In [42]:
import pandas as pd
import string
import nltk
import numpy as np
import keras
from tqdm._tqdm_notebook import tqdm_notebook
tqdm_notebook.pandas()
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import wordnet
from sklearn.feature_extraction.text import TfidfVectorizer
from keras.models import Sequential
import pickle
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential, Model
from keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation
from keras.layers.embeddings import Embedding
from keras import optimizers
from keras.layers import Input
from keras.callbacks import EarlyStopping

# Start here

In [26]:
#Save checkpoint
# data.to_csv(r"E:\Yelp\Unfiltered Data\YelpZip\Customs\checkpoint", header=True, index=False)

#Use if kernel crashes
data = pd.read_csv(r"E:\Yelp\Unfiltered Data\YelpZip\Customs\checkpoint")
data.head()

,index,user,item,review,rating,real,date
0,0,5044,0,drink be bad the hot chocolate be water down a...,1.0,-1,2014-11-16
1,1,5045,0,this be the bad experience ive ever have a cas...,1.0,-1,2014-09-08
2,2,5046,0,this be locate on the site of the old spruce s...,3.0,-1,2013-10-06
3,3,5047,0,i enjoy coffee and breakfast twice at toast du...,5.0,-1,2014-11-30
4,4,5048,0,i love toast the food choice be fantastic i lo...,5.0,-1,2014-08-28


In [ ]:
#Alternate approach

In [27]:
def binarize(n):
    if n==1:
        return 0
    else:
        return 1

data['real'] = data['real'].progress_apply(lambda n: binarize(n))
data.head()

data['review'] = data['review'].progress_apply(lambda st: str(st))

In [28]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data['review'], data['real'], test_size=0.20, random_state=42)

In [29]:
tokenizer = Tokenizer(num_words= 2000)
tokenizer.fit_on_texts(X_train)
sequences = tokenizer.texts_to_sequences(X_train)
data1 = pad_sequences(sequences, maxlen=100)

In [141]:
inputs = Input(shape=[100])
layers = Embedding(2000,50,input_length=100)(inputs)
layers = LSTM(128)(layers)
layers = Dense(256)(layers)
layers = Dense(128)(layers)
layers = Activation('relu')(layers)
layers = Dropout(0.5)(layers)
layers = Dense(1)(layers)
layers = Activation('sigmoid')(layers)
model = Model(inputs=inputs,outputs=layers)

model.summary()
model.compile(loss='binary_crossentropy',optimizer='rmsprop',metrics=['accuracy'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_8 (InputLayer)         (None, 100)               0         
_________________________________________________________________
embedding_8 (Embedding)      (None, 100, 50)           100000    
_________________________________________________________________
lstm_8 (LSTM)                (None, 128)               91648     
_________________________________________________________________
dense_6 (Dense)              (None, 256)               33024     
_________________________________________________________________
dense_7 (Dense)              (None, 128)               32896     
_________________________________________________________________
activation_15 (Activation)   (None, 128)               0         
_________________________________________________________________
dropout_8 (Dropout)          (None, 128)               0         
__________

In [142]:
model.fit(data1,y_train,batch_size=5000,epochs=10, validation_split=0.2,callbacks=[EarlyStopping(monitor='val_loss',min_delta=0.0001)])

Train on 389412 samples, validate on 97354 samples
Epoch 1/10
389412/389412 [==============================] - ETA: 11:00 - loss: 0.6920 - acc: 0.66 - ETA: 10:03 - loss: 0.6742 - acc: 0.76 - ETA: 9:30 - loss: 0.6389 - acc: 0.7975 - ETA: 9:13 - loss: 0.5740 - acc: 0.817 - ETA: 9:46 - loss: 0.5425 - acc: 0.828 - ETA: 9:42 - loss: 0.5433 - acc: 0.833 - ETA: 9:27 - loss: 0.5346 - acc: 0.838 - ETA: 9:10 - loss: 0.5192 - acc: 0.842 - ETA: 8:55 - loss: 0.5056 - acc: 0.844 - ETA: 8:41 - loss: 0.4940 - acc: 0.846 - ETA: 8:28 - loss: 0.4846 - acc: 0.848 - ETA: 8:17 - loss: 0.4768 - acc: 0.849 - ETA: 8:06 - loss: 0.4695 - acc: 0.851 - ETA: 7:56 - loss: 0.4631 - acc: 0.852 - ETA: 7:46 - loss: 0.4569 - acc: 0.854 - ETA: 7:37 - loss: 0.4521 - acc: 0.855 - ETA: 7:28 - loss: 0.4494 - acc: 0.855 - ETA: 7:19 - loss: 0.4476 - acc: 0.856 - ETA: 7:11 - loss: 0.4446 - acc: 0.856 - ETA: 7:02 - loss: 0.4422 - acc: 0.857 - ETA: 6:54 - loss: 0.4391 - acc: 0.857 - ETA: 6:46 - loss: 0.4363 - acc: 0.858 - ETA: 6:3

In [143]:
test_sequences = tokenizer.texts_to_sequences(X_test)
test_sequences_matrix = pad_sequences(test_sequences,maxlen=100)

In [144]:
# accr = model.evaluate(test_sequences_matrix,y_test)
y_pred = model.predict(test_sequences_matrix)

In [145]:
from sklearn.metrics import classification_report
# print(classification_report(y_test, y_pred))

In [173]:
attempt = []
for i in y_pred:
    if i[0] > 0.27:
        attempt.append(1)
    else:
        attempt.append(0)

In [174]:
print(classification_report(y_test, attempt))

             precision    recall  f1-score   support

          0       0.89      0.93      0.91    105656
          1       0.34      0.25      0.29     16036

avg / total       0.82      0.84      0.83    121692



In [148]:
for i in y_pred:
    print(i[0])

0.008420224
0.01090204
0.021481363
0.06908572
0.1002225
0.06760851
0.00953094
0.11272327
0.04694838
0.03721611
0.057665363
0.20084034
0.12322726
0.042165145
0.25960818
0.1697104
0.119694665
0.33332142
0.014234959
0.057106037
0.24566765
0.008189838
0.019698769
0.008071878
0.22346957
0.25676954
0.08500333
0.14524284
0.014852929
0.017399102
0.021813376
0.018786818
0.00795983
0.063743636
0.042134985
0.030973233
0.0052440427
0.14575839
0.064646095
0.25988045
0.0055531627
0.038872868
0.042472705
0.045553178
0.015422517
0.17452279
0.014157417
0.019538414
0.048631426
0.009308685
0.18523434
0.28551692
0.059223138
0.06640233
0.012275519
0.09277052
0.09113035
0.0046678092
0.2100937
0.084446646
0.043003105
0.13725558
0.025477737
0.10120738
0.20207174
0.089358926
0.095922284
0.020094234
0.0074102283
0.008058755
0.10226845
0.10766675
0.12843564
0.30785674
0.06844002
0.030934198
0.01133021
0.019597005
0.011568718
0.3880633
0.23078287
0.074337006
0.099347115
0.14916635
0.052140538
0.30416197
0.0683711

0.0739382
0.067147836
0.063045874
0.038814884
0.0067342995
0.037945878
0.3067803
0.19630978
0.012737276
0.0110518
0.009018681
0.14290188
0.0038361063
0.20636305
0.2435912
0.2886091
0.023645248
0.049258903
0.03205336
0.03405156
0.28523985
0.029244388
0.00636084
0.010587699
0.058390837
0.07420359
0.21111004
0.022783343
0.0063831024
0.010587123
0.030432044
0.054539114
0.0073876237
0.415132
0.039265335
0.049966387
0.03452118
0.3506369
0.017547833
0.29431742
0.006588083
0.003886992
0.0073951585
0.1269325
0.07155872
0.04033048
0.37627956
0.34842297
0.0060436255
0.20393164
0.009637006
0.121673316
0.3543488
0.24772064
0.2950662
0.067982644
0.025517624
0.016843375
0.15713035
0.20139836
0.08180084
0.048510294
0.0059651188
0.0074735037
0.005348596
0.21931103
0.03537546
0.18139315
0.12960544
0.0063449973
0.020579778
0.22317989
0.05740452
0.05484006
0.1839691
0.021136984
0.012829745
0.1690419
0.01971329
0.009682705
0.17138825
0.33552372
0.029237261
0.3109558
0.024294334
0.24091455
0.008092164
0.162

0.04436735
0.08859608
0.07337954
0.0046799006
0.118800096
0.03567472
0.012544477
0.06800117
0.0035920336
0.38769206
0.098364815
0.092423074
0.18631035
0.0076076835
0.014463728
0.13286053
0.01608092
0.026600285
0.13404104
0.17996593
0.3831322
0.09556239
0.009915786
0.3486699
0.09436177
0.17683132
0.12652405
0.1575383
0.06863062
0.051393025
0.03024328
0.073033676
0.041110057
0.031703185
0.014070855
0.02445453
0.005663186
0.046268128
0.17176767
0.004708817
0.015868422
0.13057803
0.010227134
0.008920506
0.1360961
0.27402562
0.23266758
0.0045793755
0.09849197
0.008879446
0.33335304
0.007026352
0.05796783
0.0042435895
0.09750756
0.1836653
0.0062837475
0.024469174
0.021345885
0.16925082
0.3830785
0.010233382
0.07785533
0.047029257
0.18385658
0.07372948
0.08326704
0.05472451
0.06530186
0.13792397
0.08808948
0.007148273
0.006045963
0.02055514
0.022115717
0.0393359
0.085480206
0.008565115
0.006365305
0.3203383
0.106095575
0.1316791
0.03138503
0.035968497
0.13426474
0.3632024
0.39947578
0.0577774

0.015756592
0.31720072
0.03403417
0.08952131
0.005408003
0.13283668
0.007997023
0.24970204
0.009565817
0.34966567
0.08372459
0.030750873
0.19075145
0.019255368
0.04267849
0.051138874
0.021390155
0.012604215
0.079932556
0.008382498
0.025744833
0.0622915
0.06449628
0.044303726
0.2369674
0.0057571153
0.03703734
0.022033995
0.005815787
0.030980727
0.02839121
0.004688302
0.0100575425
0.03668949
0.05512484
0.14119035
0.17742892
0.005275903
0.06774495
0.008581091
0.077354945
0.17369562
0.2375428
0.11874998
0.03834561
0.010894094
0.021544384
0.008436164
0.014077843
0.04918001
0.005021774
0.030716797
0.075574845
0.04455317
0.2463062
0.2552157
0.053259954
0.23088472
0.053115495
0.007709606
0.058729213
0.0067452798
0.06262877
0.03344071
0.16586556
0.054456364
0.014234846
0.07710674
0.33048922
0.22765532
0.014199767
0.38347754
0.012640497
0.37471813
0.11274779
0.020352034
0.006466767
0.23210189
0.31665215
0.023714365
0.05668141
0.08740052
0.04261786
0.030159015
0.0076763104
0.1258465
0.16799055
0.

0.007752217
0.0099579925
0.0060420614
0.024702042
0.20613873
0.020058926
0.106908664
0.15629084
0.025171978
0.2916166
0.0052169967
0.030751904
0.239403
0.011468003
0.04770403
0.314376
0.08516285
0.20782585
0.12960258
0.017315708
0.07124399
0.02090737
0.20824358
0.024464542
0.016623808
0.04594778
0.04873419
0.02162089
0.04896295
0.012710962
0.03490633
0.05765549
0.0059237555
0.010279065
0.19644907
0.36802432
0.041002993
0.0397841
0.23325418
0.09838507
0.07841563
0.11763259
0.03832998
0.07088069
0.083700955
0.012732577
0.009622981
0.018799681
0.38518408
0.23541139
0.34236032
0.11961374
0.12530136
0.032272708
0.23124129
0.18630974
0.009443169
0.0459986
0.28469113
0.3995666
0.0066461065
0.15100215
0.06974503
0.010378363
0.09668916
0.01636914
0.007131024
0.032017156
0.004853959
0.057076108
0.26191068
0.01403706
0.072417006
0.15148522
0.15031339
0.004284559
0.37012503
0.0074263196
0.10678645
0.038658492
0.0072388435
0.013439321
0.09620364
0.029934566
0.13969068
0.01495415
0.006551869
0.00747

...

0.063235424
0.029153032
0.013192091
0.021997394
0.388278
0.007061809
0.096910186
0.15869527
0.21055232
0.074336216
0.1292779
0.096879475
0.21263334
0.08260443
0.10241095
0.33054918
0.07867936
0.12541248
0.010137841
0.048839852
0.055199698
0.005733184
0.23362494
0.02017074
0.00711158
0.03946747
0.054539483
0.0057533113
0.03723585
0.23028322
0.009894891
0.039000094
0.30774716
0.013222641
0.015410979
0.30791524
0.071761884
0.06645908
0.029364483
0.2767902
0.13730517
0.25142217
0.11323519
0.1867165
0.27157333
0.16912235
0.0076969434
0.010790323
0.14794266
0.07432785
0.18244798
0.0037360855
0.17776327
0.037257142
0.009482976
0.09904741
0.00897437
0.00606598
0.21297988
0.026820213
0.016100593
0.32720235
0.004453533
0.15338963
0.012556018
0.14417197
0.020608954
0.037371017
0.008954566
0.056528483
0.008856785
0.124466114
0.103631206
0.03803414
0.003020424
0.11689527
0.35929886
0.1469951
0.09561728
0.15999013
0.08533086
0.044841982
0.017582392
0.18105668
0.1603632
0.0575902
0.006599821
0.013117

0.016748307
0.015055931
0.26472768
0.009065038
0.005024889
0.032025922
0.12086061
0.011897491
0.00711544
0.005001504
0.12727687
0.21999952
0.1491861
0.052352283
0.016167816
0.07899658
0.05717203
0.008721079
0.1346486
0.13415825
0.0045141513
0.01631113
0.011665517
0.27574354
0.03302516
0.054586444
0.39418942
0.09869462
0.004283904
0.36733305
0.2988841
0.008205102
0.23085338
0.07751839
0.17521885
0.15060396
0.054880176
0.22313266
0.079269886
0.23333283
0.007062979
0.034244206
0.007422532
0.2682809
0.0115196435
0.008384659
0.3184281
0.025285315
0.01941529
0.077808775
0.006564147
0.10840187
0.013962996
0.11210094
0.08415
0.018653365
0.0048711724
0.036521845
0.018725438
0.0060423394
0.06850814
0.058661688
0.25886542
0.008181921
0.0060292743
0.04292027
0.24698041
0.22851843
0.02546674
0.38682306
0.013319751
0.14177169
0.013399776
0.12842745
0.06570774
0.06644978
0.023113655
0.041453872
0.0074195676
0.08261498
0.0067220693
0.017252637
0.36059636
0.1029476
0.31673953
0.015953686
0.21210675
0.0

0.081091374
0.081659
0.06780557
0.40857178
0.09959603
0.009317923
0.21726699
0.3614682
0.42809045
0.31177878
0.31604818
0.029765638
0.016827581
0.010246846
0.08369161
0.28662705
0.028529407
0.095330976
0.2725025
0.24248064
0.21603224
0.07649999
0.010286762
0.022249581
0.0033359192
0.0071450183
0.0047037983
0.025306607
0.03545369
0.39236256
0.007961208
0.045405574
0.2536869
0.3819489
0.22205049
0.13115261
0.025859533
0.17964002
0.0049768826
0.24649908
0.006666288
0.123406276
0.005310804
0.34435117
0.27037048
0.0060608136
0.046126053
0.1833255
0.2800432
0.1309858
0.05930599
0.096912175
0.14285928
0.014654736
0.02199934
0.052084785
0.17974599
0.00869888
0.019944744
0.008503106
0.22715734
0.09670157
0.031489212
0.01059202
0.10889534
0.12428406
0.0043323347
0.020880574
0.0047378754
0.17518117
0.12955146
0.03870213
0.19555148
0.11036284
0.15939042
0.11364439
0.009638139
0.0098832
0.16358618
0.005127053
0.23965189
0.011559364
0.1990602
0.006501556
0.25934917
0.007773512
0.035467748
0.01820789

0.14334387
0.10326483
0.13670892
0.034328762
0.0076268413
0.19325279
0.015649583
0.017699677
0.016904077
0.0065452
0.037051328
0.007899616
0.18650989
0.007908941
0.22157137
0.45132926
0.039535474
0.41944373
0.34475228
0.12571211
0.24061087
0.06531002
0.01874026
0.061201952
0.05162962
0.01536786
0.020996986
0.36044273
0.033414926
0.04104995
0.28800437
0.13679685
0.018257657
0.10305702
0.21119682
0.2617618
0.052571826
0.117193975
0.010863461
0.40745965
0.020069674
0.018237984
0.07920492
0.006098345
0.02191499
0.33213988
0.09719405
0.016221376
0.007476231
0.049296826
0.0044486206
0.24778059
0.008402686
0.008451391
0.14285026
0.016389644
0.020745415
0.014644045
0.22836648
0.019660268
0.010861227
0.007436921
0.29858395
0.0050604306
0.014601318
0.15434505
0.020543685
0.21206321
0.26576254
0.012222246
0.18165424
0.10197538
0.19130248
0.006202021
0.0077506946
0.12156591
0.007876963
0.20902608
0.027459916
0.0054794587
0.28821987
0.1437735
0.05348368
0.014685069
0.022347398
0.31591815
0.1812208


0.11536112
0.02696507
0.19540979
0.1342318
0.07200742
0.24241471
0.022969732
0.007965337
0.0144618265
0.34725645
0.12847517
0.038981456
0.0077030174
0.009754074
0.0057298825
0.337247
0.2108996
0.010426605
0.06947441
0.011746184
0.0056308154
0.021215482
0.07892297
0.05079686
0.08669689
0.32005057
0.023637203
0.08181001
0.014801663
0.018388879
0.03196748
0.019640965
0.14044635
0.08746714
0.066390306
0.009151947
0.22828306
0.16563857
0.2394489
0.17191572
0.051131852
0.03644118
0.25840628
0.005318094
0.026196577
0.018473031
0.010928862
0.017758586
0.016230403
0.12712629
0.005573682
0.0445224
0.12003338
0.0116516
0.08874416
0.037950646
0.20938684
0.16239062
0.24116649
0.30805913
0.05947572
0.012224964
0.12107569
0.12468979
0.0079633845
0.012981648
0.061394423
0.018452473
0.0042569693
0.005197747
0.01248466
0.0094209025
0.23683475
0.012816231
0.17357226
0.0070336214
0.09432738
0.007317207
0.0047020973
0.013501254
0.097694546
0.11002572
0.018698646
0.11563817
0.113267116
0.007836864
0.0193875